# Social Good Hackathon - Covid19 and Texas Counties

# Data Prep Part 1

#### Files used (11)
2020 Texas County Health.csv  
Age_final.csv  
education_unemployment_final.csv  
ethnicity_race_final.csv  
new policitcal demographics_final.csv  
Texas Nursing Facility.csv  
texas prisons.csv  
TX hospital beds.csv  
TX military installations.csv  
TxDOT_Roadway_Inventory.txt  
Texas detail age.csv  
Texas_county_numbers.csv

In [163]:
import pandas as pd

In [164]:
import sagemaker.amazon.common as smac
import sagemaker
from sagemaker import get_execution_role

In [165]:
role = get_execution_role()
input_bucket = 'texas-counties-covid19'

## First set of features

In [166]:
# dataset originally downloaded from:  https://imis.county.org/iMIS/CountyInformationProgram/QueriesCIP.aspx
in_data_key = 'Age_final.csv'
data_location = 's3://{}/{}'.format(input_bucket, in_data_key)
age = pd.read_csv(data_location)

In [167]:
# dataset originally downloaded from:  https://imis.county.org/iMIS/CountyInformationProgram/QueriesCIP.aspx
in_data_key = "education_unemployment_final.csv"
data_location = 's3://{}/{}'.format(input_bucket, in_data_key)
edu = pd.read_csv(data_location)

In [168]:
# dataset originally downloaded from:  https://imis.county.org/iMIS/CountyInformationProgram/QueriesCIP.aspx
in_data_key = "ethnicity_race_final.csv"
data_location = 's3://{}/{}'.format(input_bucket, in_data_key)
race = pd.read_csv(data_location)

In [169]:
# dataset originally downloaded from:  https://imis.county.org/iMIS/CountyInformationProgram/QueriesCIP.aspx
in_data_key = "new policitcal demographics_final.csv"
data_location = 's3://{}/{}'.format(input_bucket, in_data_key)
poli = pd.read_csv(data_location)

In [170]:
# dataset originally downloaded from:  
# https://www.txdot.gov/inside-txdot/division/transportation-planning/roadway-inventory.html
in_data_key="TxDOT_Roadway_Inventory.txt"
data_location = 's3://{}/{}'.format(input_bucket, in_data_key)
roads = pd.read_csv(data_location, sep="|", usecols=[30, 208, 209, 210, 211])
roads.head()

,CO,LEN_SEC,LN_MILES,DVMT,DTRKVMT
0,15,0.416,0.416,401.024,0.000
1,15,0.123,0.123,1043.532,40.713
2,15,0.006,0.006,50.904,1.986
3,15,0.279,0.279,2367.036,92.349
4,15,0.326,0.326,3871.902,0.000


In [171]:
# selection pertinent features from the larger roads dataset
#small_roads = roads.iloc[:, [30, 208, 209, 210, 211]]
small_roads = roads.copy()

In [172]:
roads_by_county = small_roads.groupby("CO").sum()

In [173]:
roads_by_county.reset_index(inplace=True)

In [174]:
roads_by_county

,CO,LEN_SEC,LN_MILES,DVMT,DTRKVMT
0,1,1563.054,3241.985,1771161.086,217583.748
1,2,681.348,1440.703,1817432.214,340429.090
2,3,1522.123,3226.201,4012089.437,747829.119
3,4,465.179,916.157,937527.224,79097.644
4,5,782.152,1608.504,604312.790,110647.681
...,...,...,...,...,...
249,250,1341.465,2756.760,1246402.933,120880.396
250,251,750.011,1515.201,402569.649,55363.059
251,252,1011.298,2041.825,486813.968,56522.578
252,253,299.189,634.108,369961.028,54823.312


In [175]:
# the df translates the county numbers used in roads_by_county df to county names
in_data_key = "Texas_county_numbers.csv"
data_location = 's3://{}/{}'.format(input_bucket, in_data_key)
co_num = pd.read_csv(data_location)

In [176]:
roads_names = roads_by_county.merge(co_num, how="left", on="CO")
roads_names.drop("CO", axis=1, inplace=True)
roads_names

,LEN_SEC,LN_MILES,DVMT,DTRKVMT,County
0,1563.054,3241.985,1771161.086,217583.748,Anderson
1,681.348,1440.703,1817432.214,340429.090,Andrews
2,1522.123,3226.201,4012089.437,747829.119,Angelina
3,465.179,916.157,937527.224,79097.644,Aransas
4,782.152,1608.504,604312.790,110647.681,Archer
...,...,...,...,...,...
250,1341.465,2756.760,1246402.933,120880.396,Wood
251,750.011,1515.201,402569.649,55363.059,Yoakum
252,1011.298,2041.825,486813.968,56522.578,Young
253,299.189,634.108,369961.028,54823.312,Zapata


In [177]:
age.head()

,County,Median Age,% Age 17 and Under,% Age 65 and Older,% Age 85 and Older
0,Anderson,39.1,19.58,14.54,1.48
1,Andrews,31.8,30.90,10.53,1.37
2,Angelina,37.1,25.81,15.79,1.94
3,Aransas,49.3,18.73,27.30,2.62
4,Archer,44.3,21.77,19.89,2.17


In [178]:
edu.head()

,County,% High School Graduate or Higher,% Bachelor s Degree or Higher,Unemployment Rate (%)
0,Anderson,80.2,11.8,3.6
1,Andrews,73.8,10.6,3.1
2,Angelina,79.9,15.7,5.1
3,Aransas,83.2,20.2,6.8
4,Archer,90.1,21.8,3.4


In [179]:
race.head()

,County,Ethnicity: % Hispanic,Race: % White Alone,Race: % African American Alone,Race: % American Indian & Alaska Native Alone,Race: % Asian Alone,Race: % Native Hawaiian and Other Pacific Islander Alone,Race: % Multi-Racial
0,Anderson,17.82,75.11,21.49,0.68,0.90,0.14,1.67
1,Andrews,56.19,94.22,1.94,1.47,0.75,0.02,1.61
2,Angelina,22.18,81.27,15.44,0.75,1.17,0.06,1.31
3,Aransas,27.66,93.05,1.79,1.24,1.93,0.08,1.90
4,Archer,8.68,95.58,1.03,1.42,0.43,0.03,1.50


In [180]:
poli.head()

,County,Votes,Percent_Voted,Republicans 2016,Democrats 2016,Green 2016,Libertarians 2016,Preschool.Enrollment.Ratio.enrolled.ages.3.and.4,Gini.Coefficient,Management.professional.and.related.occupations,...,Violent.crime,Injury.deaths,CA,S,MAR,CFS,ACFS,Mean Alc,Max Alc,Mixedness
0,Anderson,16887,0.293605,77.959377,19.885119,0.337538,1.817966,35.9,0.465,23.60,...,311.56,93.1,0.229051,-0.948391,0.711662,0.000005,0.000005,0.000074,0.000423,-0.470898
1,Andrews,4926,0.350855,79.679253,16.971173,0.365408,2.984166,38.4,0.531,20.90,...,536.37,79.8,0.827519,-1.081741,0.821886,0.000045,0.000045,0.000109,0.000622,-0.052599
2,Angelina,29870,0.352998,72.534315,25.236023,0.354871,1.874791,51.3,0.452,27.75,...,350.84,65.2,0.620367,-0.793494,0.479538,0.000029,0.000029,0.000060,0.000468,-0.681732
3,Aransas,10467,0.439273,73.851151,23.483329,0.391707,2.273813,67.9,0.470,31.05,...,186.09,102.4,-0.093289,-0.794147,0.733924,0.000044,0.000044,0.000089,0.000353,-0.216679
4,Archer,4269,0.474597,88.662450,9.229328,0.234247,1.873975,62.9,0.409,28.20,...,112.93,59.6,1.421798,0.230079,0.547123,0.000008,0.000008,0.000081,0.000426,-0.653378


In [181]:
# merging all the datasets together 
final_df = age.merge(edu, how="left", on="County")

In [182]:
final_df = final_df.merge(race, how="left", on="County")

In [183]:
final_df = final_df.merge(poli, how="left", on="County")

In [184]:
final_df = final_df.merge(roads_names, how="left", on="County")

In [185]:
final_df

,County,Median Age,% Age 17 and Under,% Age 65 and Older,% Age 85 and Older,% High School Graduate or Higher,% Bachelor s Degree or Higher,Unemployment Rate (%),Ethnicity: % Hispanic,Race: % White Alone,...,MAR,CFS,ACFS,Mean Alc,Max Alc,Mixedness,LEN_SEC,LN_MILES,DVMT,DTRKVMT
0,Anderson,39.1,19.58,14.54,1.48,80.2,11.8,3.6,17.82,75.11,...,0.711662,0.000005,0.000005,0.000074,0.000423,-0.470898,1563.054,3241.985,1771161.086,217583.748
1,Andrews,31.8,30.90,10.53,1.37,73.8,10.6,3.1,56.19,94.22,...,0.821886,0.000045,0.000045,0.000109,0.000622,-0.052599,681.348,1440.703,1817432.214,340429.090
2,Angelina,37.1,25.81,15.79,1.94,79.9,15.7,5.1,22.18,81.27,...,0.479538,0.000029,0.000029,0.000060,0.000468,-0.681732,1522.123,3226.201,4012089.437,747829.119
3,Aransas,49.3,18.73,27.30,2.62,83.2,20.2,6.8,27.66,93.05,...,0.733924,0.000044,0.000044,0.000089,0.000353,-0.216679,465.179,916.157,937527.224,79097.644
4,Archer,44.3,21.77,19.89,2.17,90.1,21.8,3.4,8.68,95.58,...,0.547123,0.000008,0.000008,0.000081,0.000426,-0.653378,782.152,1608.504,604312.790,110647.681
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,Wood,48.7,19.34,27.10,2.82,85.1,17.1,4.7,10.07,91.42,...,0.690706,0.000039,0.000039,0.000041,0.000152,-0.402126,1341.465,2756.760,1246402.933,120880.396
250,Yoakum,30.8,33.03,11.67,1.31,71.4,17.8,3.5,66.57,95.05,...,1.018043,0.000112,0.000112,0.000159,0.000614,0.652742,750.011,1515.201,402569.649,55363.059
251,Young,41.4,24.14,20.01,2.69,81.6,18.6,3.8,18.95,95.17,...,0.878403,0.000042,0.000042,0.000073,0.000268,-0.073630,1011.298,2041.825,486813.968,56522.578
252,Zapata,29.7,33.29,12.57,1.39,50.3,8.3,7.3,94.44,98.55,...,1.810431,0.000210,0.000210,0.000641,0.001929,3.248312,299.189,634.108,369961.028,54823.312


In [186]:
# examining the final_df for null values
final_df.isna().sum()

County                                                         0
Median Age                                                     0
% Age 17 and Under                                             0
% Age 65 and Older                                             0
% Age 85 and Older                                             0
% High School Graduate or Higher                               0
% Bachelor s Degree or Higher                                  0
Unemployment Rate (%)                                          0
Ethnicity: % Hispanic                                          0
Race: % White Alone                                            0
Race: % African American Alone                                 0
Race: % American Indian & Alaska Native Alone                  0
Race: % Asian Alone                                            0
Race: % Native Hawaiian and Other Pacific Islander Alone       0
Race: % Multi-Racial                                           0
Votes                    

In [187]:
# fill in all nulls with the median of the column 
#final_df_fillna = final_df.iloc[:, 1:].apply(lambda col: col.fillna(col.median()), axis=0)

In [188]:
final_df_fillna["County"] = final_df["County"]

In [189]:
final_df_fillna.isna().sum()

Median Age                                                    0
% Age 17 and Under                                            0
% Age 65 and Older                                            0
% Age 85 and Older                                            0
% High School Graduate or Higher                              0
% Bachelor s Degree or Higher                                 0
Unemployment Rate (%)                                         0
Ethnicity: % Hispanic                                         0
Race: % White Alone                                           0
Race: % African American Alone                                0
Race: % American Indian & Alaska Native Alone                 0
Race: % Asian Alone                                           0
Race: % Native Hawaiian and Other Pacific Islander Alone      0
Race: % Multi-Racial                                          0
Votes                                                         0
Percent_Voted                           

In [190]:
final_df_fillna

,Median Age,% Age 17 and Under,% Age 65 and Older,% Age 85 and Older,% High School Graduate or Higher,% Bachelor s Degree or Higher,Unemployment Rate (%),Ethnicity: % Hispanic,Race: % White Alone,Race: % African American Alone,...,CFS,ACFS,Mean Alc,Max Alc,Mixedness,LEN_SEC,LN_MILES,DVMT,DTRKVMT,County
0,39.1,19.58,14.54,1.48,80.2,11.8,3.6,17.82,75.11,21.49,...,0.000005,0.000005,0.000074,0.000423,-0.470898,1563.054,3241.985,1771161.086,217583.748,Anderson
1,31.8,30.90,10.53,1.37,73.8,10.6,3.1,56.19,94.22,1.94,...,0.000045,0.000045,0.000109,0.000622,-0.052599,681.348,1440.703,1817432.214,340429.090,Andrews
2,37.1,25.81,15.79,1.94,79.9,15.7,5.1,22.18,81.27,15.44,...,0.000029,0.000029,0.000060,0.000468,-0.681732,1522.123,3226.201,4012089.437,747829.119,Angelina
3,49.3,18.73,27.30,2.62,83.2,20.2,6.8,27.66,93.05,1.79,...,0.000044,0.000044,0.000089,0.000353,-0.216679,465.179,916.157,937527.224,79097.644,Aransas
4,44.3,21.77,19.89,2.17,90.1,21.8,3.4,8.68,95.58,1.03,...,0.000008,0.000008,0.000081,0.000426,-0.653378,782.152,1608.504,604312.790,110647.681,Archer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,48.7,19.34,27.10,2.82,85.1,17.1,4.7,10.07,91.42,5.41,...,0.000039,0.000039,0.000041,0.000152,-0.402126,1341.465,2756.760,1246402.933,120880.396,Wood
250,30.8,33.03,11.67,1.31,71.4,17.8,3.5,66.57,95.05,1.41,...,0.000112,0.000112,0.000159,0.000614,0.652742,750.011,1515.201,402569.649,55363.059,Yoakum
251,41.4,24.14,20.01,2.69,81.6,18.6,3.8,18.95,95.17,1.52,...,0.000042,0.000042,0.000073,0.000268,-0.073630,1011.298,2041.825,486813.968,56522.578,Young
252,29.7,33.29,12.57,1.39,50.3,8.3,7.3,94.44,98.55,0.38,...,0.000210,0.000210,0.000641,0.001929,3.248312,299.189,634.108,369961.028,54823.312,Zapata


## Second set of features

In [191]:
# dataset originally downloaded from: https://data.medicare.gov/Nursing-Home-Compare/Provider-Info/4pq5-n9py
in_data_key = "Texas Nursing Facility.csv"
data_location = 's3://{}/{}'.format(input_bucket, in_data_key)
nursing_homes = pd.read_csv(data_location)

In [192]:
nursing_homes.head()

,county,nursing home capacity
0,ANDERSON,98
1,ANDERSON,120
2,ANDERSON,199
3,ANDERSON,102
4,ANDERSON,120


In [193]:
nh_df = nursing_homes.groupby("county").sum().reset_index()
nh_df.head()

,county,nursing home capacity
0,ANDERSON,639
1,ANDREWS,90
2,ANGELINA,920
3,ARANSAS,212
4,ARCHER,46


In [194]:
nh_df["county"] = nh_df["county"].apply(lambda x: x.title())

In [195]:
nh_df.head()

,county,nursing home capacity
0,Anderson,639
1,Andrews,90
2,Angelina,920
3,Aransas,212
4,Archer,46


In [196]:
# dataset originally downloaded from: https://texasalmanac.com/topics/government/texas-prisons
in_data_key = "texas prisons.csv"
data_location = 's3://{}/{}'.format(input_bucket, in_data_key)
prisons = pd.read_csv(data_location)

In [197]:
prisons.head()

,county,max capacity (gender),employees
0,Anderson,"3,471 Male",633
1,Anderson,"4,139 Male",879
2,Anderson,"2,128 Male",437
3,Anderson,"3,800 Male",816
4,Anderson,"1,137 Male",290


In [198]:
prisons["capacity"] = prisons["max capacity (gender)"].apply(lambda x: x.split()[0].replace(',', ''))
prisons["employees"] = prisons["employees"].apply(lambda x: x.replace(',', ''))
prisons.head()

,county,max capacity (gender),employees,capacity
0,Anderson,"3,471 Male",633,3471
1,Anderson,"4,139 Male",879,4139
2,Anderson,"2,128 Male",437,2128
3,Anderson,"3,800 Male",816,3800
4,Anderson,"1,137 Male",290,1137


In [199]:
prisons.iloc[:, 2:4] = prisons.iloc[:, 2:4].apply(lambda x: x.astype('int64'))
prisons["prison capacity and jail employees"] = prisons["employees"] + prisons["capacity"]

In [200]:
prisons.drop(columns=["max capacity (gender)", "employees", "capacity"], axis=1, inplace=True)

In [201]:
prisons.head()

,county,prison capacity and jail employees
0,Anderson,4104
1,Anderson,5018
2,Anderson,2565
3,Anderson,4616
4,Anderson,1427


In [202]:
prisons_df = prisons.groupby("county").sum()
prisons_df.head()

,prison capacity and jail employees
county,
Anderson,17730
Angelina,1399
Bee,9021
Bexar,2658
Bowie,3578


In [203]:
# dataset originally downloaded from: https://texasalmanac.com/topics/government/major-military-installations
in_data_key = "TX military installations.csv"
data_location = 's3://{}/{}'.format(input_bucket, in_data_key)
military = pd.read_csv(data_location)
military.head()

,county,military installation
0,Nueces,1
1,Tarrant,1
2,Kleberg,1
3,El Paso,1
4,Bell,1


In [204]:
military.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   county                 11 non-null     object
 1   military installation  11 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 304.0+ bytes


In [205]:
# dataset originally downloaded from:  https://hifld-geoplatform.opendata.arcgis.com/datasets/6ac5e325468c4cb9b905f1728d6fbf0f_0
in_data_key = "TX hospital beds.csv"
data_location = 's3://{}/{}'.format(input_bucket, in_data_key)
hos_beds = pd.read_csv(data_location)

In [206]:
hos_beds.head()

,county,hospital beds
0,ANDERSON,86
1,ANDERSON,70
2,ANDREWS,34
3,ANGELINA,149
4,ANGELINA,24


In [207]:
hos_beds_df = hos_beds.groupby(["county"]).sum().reset_index()
hos_beds_df["county"] = hos_beds_df["county"].apply(lambda x: x.title())
hos_beds_df.head()

,county,hospital beds
0,Anderson,156
1,Andrews,34
2,Angelina,470
3,Atascosa,67
4,Austin,32


In [208]:
# dataset originally downloaded from:  https://demographics.texas.gov/Download/Estimates?fid=b6b495fa6c904a318b184d21eedf860b)
in_data_key = "Texas detail age.csv"
data_location = 's3://{}/{}'.format(input_bucket, in_data_key)
detail_age = pd.read_csv(data_location)
detail_age.head()

,county,Age range,population
0,Anderson,< 10,"5,816"
1,Anderson,19-Oct,"6,633"
2,Anderson,20-29,"8,198"
3,Anderson,30-39,"9,904"
4,Anderson,40-49,"8,920"


In [209]:
detail_age_df = detail_age.groupby('county')['population'].apply(lambda df: df.reset_index(drop=True)).unstack().reset_index()
detail_age_df.head()

,county,0,1,2,3,4,5,6,7,8,9
0,Anderson,"5,816","6,633","8,198","9,904","8,920","7,433","5,980","3,963","1,775",357
1,Andrews,"3,169","3,044","2,912","2,863","2,088","1,912","1,431",753,414,92
2,Angelina,"12,714","13,386","12,292","11,347","10,915","11,311","9,583","6,451","3,028",660
3,Aransas,"2,416","2,431","2,646","2,507","2,321","3,093","3,696","3,149","1,243",222
4,Archer,"1,008","1,180","1,176",984,"1,094","1,383","1,325",814,423,72


In [210]:
detail_age_df.rename(columns={0: "% lt_10", 1: "% 10_19", 2: "% 20_29", 3: "% 30_39", 4: "% 40_49", 5: "% 50-59", 6: "% 60_69", 7: "% 70_79", 8: "% 80_89", 9: "% over_90"}, inplace=True)
detail_age_df.head()

,county,% lt_10,% 10_19,% 20_29,% 30_39,% 40_49,% 50-59,% 60_69,% 70_79,% 80_89,% over_90
0,Anderson,"5,816","6,633","8,198","9,904","8,920","7,433","5,980","3,963","1,775",357
1,Andrews,"3,169","3,044","2,912","2,863","2,088","1,912","1,431",753,414,92
2,Angelina,"12,714","13,386","12,292","11,347","10,915","11,311","9,583","6,451","3,028",660
3,Aransas,"2,416","2,431","2,646","2,507","2,321","3,093","3,696","3,149","1,243",222
4,Archer,"1,008","1,180","1,176",984,"1,094","1,383","1,325",814,423,72


In [211]:
detail_age_df = detail_age_df.applymap(lambda x: x.replace(",", ''))
detail_age_df.head()

,county,% lt_10,% 10_19,% 20_29,% 30_39,% 40_49,% 50-59,% 60_69,% 70_79,% 80_89,% over_90
0,Anderson,5816,6633,8198,9904,8920,7433,5980,3963,1775,357
1,Andrews,3169,3044,2912,2863,2088,1912,1431,753,414,92
2,Angelina,12714,13386,12292,11347,10915,11311,9583,6451,3028,660
3,Aransas,2416,2431,2646,2507,2321,3093,3696,3149,1243,222
4,Archer,1008,1180,1176,984,1094,1383,1325,814,423,72


In [212]:
detail_age_df.iloc[:, 1:11] = detail_age_df.iloc[:, 1:11].apply(lambda x: x.astype('int64'))
detail_age_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 254 entries, 0 to 253
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   county     254 non-null    object
 1   % lt_10    254 non-null    object
 2   % 10_19    254 non-null    object
 3   % 20_29    254 non-null    object
 4   % 30_39    254 non-null    object
 5   % 40_49    254 non-null    object
 6   % 50-59    254 non-null    object
 7   % 60_69    254 non-null    object
 8   % 70_79    254 non-null    object
 9   % 80_89    254 non-null    object
 10  % over_90  254 non-null    object
dtypes: object(11)
memory usage: 22.0+ KB


In [213]:
detail_age_df["Population_total"] = detail_age_df.iloc[:, 1:11].sum(axis=1)
detail_age_df.head()

,county,% lt_10,% 10_19,% 20_29,% 30_39,% 40_49,% 50-59,% 60_69,% 70_79,% 80_89,% over_90,Population_total
0,Anderson,5816,6633,8198,9904,8920,7433,5980,3963,1775,357,58979.0
1,Andrews,3169,3044,2912,2863,2088,1912,1431,753,414,92,18678.0
2,Angelina,12714,13386,12292,11347,10915,11311,9583,6451,3028,660,91687.0
3,Aransas,2416,2431,2646,2507,2321,3093,3696,3149,1243,222,23724.0
4,Archer,1008,1180,1176,984,1094,1383,1325,814,423,72,9459.0


In [214]:
detail_age_df.iloc[:, 1:11] = detail_age_df.iloc[:, 1:11].div(detail_age_df["Population_total"], axis=0)


In [215]:
detail_age_df.drop(columns=["Population_total"], inplace=True)
detail_age_df

,county,% lt_10,% 10_19,% 20_29,% 30_39,% 40_49,% 50-59,% 60_69,% 70_79,% 80_89,% over_90
0,Anderson,0.0986114,0.112464,0.138999,0.167924,0.15124,0.126028,0.101392,0.0671934,0.0300955,0.006053
1,Andrews,0.169665,0.162972,0.155905,0.153282,0.111789,0.102366,0.0766142,0.0403148,0.0221651,0.00492558
2,Angelina,0.138667,0.145997,0.134065,0.123758,0.119046,0.123365,0.104519,0.0703589,0.0330254,0.0071984
3,Aransas,0.101838,0.10247,0.111533,0.105674,0.0978334,0.130374,0.155792,0.132735,0.0523942,0.00935761
4,Archer,0.106565,0.124749,0.124326,0.104028,0.115657,0.14621,0.140078,0.0860556,0.0447193,0.0076118
...,...,...,...,...,...,...,...,...,...,...,...
249,Wood,0.108236,0.112971,0.104168,0.0978771,0.0944537,0.130866,0.153229,0.130177,0.0569968,0.0110259
250,Yoakum,0.178651,0.167787,0.134633,0.123096,0.101703,0.114023,0.0875896,0.0581317,0.0281138,0.0062724
251,Young,0.136209,0.121399,0.109832,0.116156,0.105346,0.125399,0.132858,0.094103,0.0468083,0.0118912
252,Zapata,0.184873,0.168034,0.13721,0.126151,0.112808,0.0993935,0.0823403,0.0603639,0.0224046,0.00642169


In [216]:
# dataset originally downloaded from:  https://www.countyhealthrankings.org/app/texas/2020/downloads
in_data_key = "2020 Texas County Health.csv"
data_location = 's3://{}/{}'.format(input_bucket, in_data_key)
health_data = pd.read_csv(data_location)
health_data.head()

,county,Life Expectancy,Age-Adjusted Death Rate,Motor Vehicle Mortality Rate,% Insufficient Sleep,% Enrolled in Free or Reduced Lunch,Segregation Index,% Homeowners,Years of Potential Life Lost Rate,% Fair or Poor Health,% Smokers,% Physically Inactive,% With Access to Exercise Opportunities,% Flu Vaccinated,High School Graduation Rate,Income inequality,Social Association Rate,Average Daily PM2.5
0,Anderson,73.4,569.0,20.0,35,59.0,31.0,71,10927.0,20,17,23,26,43.0,97.0,4.3,10.0,9.9
1,Andrews,77.5,372.0,39.0,32,40.0,10.0,73,7138.0,20,13,26,94,24.0,96.0,4.3,7.3,7.3
2,Angelina,76.1,462.0,19.0,35,67.0,37.0,66,8914.0,21,16,35,65,48.0,92.0,4.4,11.3,9.8
3,Aransas,77.9,444.0,20.0,30,96.0,21.0,78,8770.0,21,14,36,81,42.0,89.0,5.3,5.1,9.3
4,Archer,78.8,384.0,29.0,29,29.0,27.0,84,8253.0,14,14,19,23,46.0,99.0,4.4,9.1,8.5


In [217]:
# merging the second set of datasets together
final2_df = detail_age_df.merge(nh_df, how="left", on="county")
final2_df.head()

,county,% lt_10,% 10_19,% 20_29,% 30_39,% 40_49,% 50-59,% 60_69,% 70_79,% 80_89,% over_90,nursing home capacity
0,Anderson,0.0986114,0.112464,0.138999,0.167924,0.15124,0.126028,0.101392,0.0671934,0.0300955,0.006053,639.0
1,Andrews,0.169665,0.162972,0.155905,0.153282,0.111789,0.102366,0.0766142,0.0403148,0.0221651,0.00492558,90.0
2,Angelina,0.138667,0.145997,0.134065,0.123758,0.119046,0.123365,0.104519,0.0703589,0.0330254,0.0071984,920.0
3,Aransas,0.101838,0.10247,0.111533,0.105674,0.0978334,0.130374,0.155792,0.132735,0.0523942,0.00935761,212.0
4,Archer,0.106565,0.124749,0.124326,0.104028,0.115657,0.14621,0.140078,0.0860556,0.0447193,0.0076118,46.0


In [218]:
final3_df = final2_df.merge(prisons_df, how="left", on="county")
final3_df.head()

,county,% lt_10,% 10_19,% 20_29,% 30_39,% 40_49,% 50-59,% 60_69,% 70_79,% 80_89,% over_90,nursing home capacity,prison capacity and jail employees
0,Anderson,0.0986114,0.112464,0.138999,0.167924,0.15124,0.126028,0.101392,0.0671934,0.0300955,0.006053,639.0,17730.0
1,Andrews,0.169665,0.162972,0.155905,0.153282,0.111789,0.102366,0.0766142,0.0403148,0.0221651,0.00492558,90.0,NaN
2,Angelina,0.138667,0.145997,0.134065,0.123758,0.119046,0.123365,0.104519,0.0703589,0.0330254,0.0071984,920.0,1399.0
3,Aransas,0.101838,0.10247,0.111533,0.105674,0.0978334,0.130374,0.155792,0.132735,0.0523942,0.00935761,212.0,NaN
4,Archer,0.106565,0.124749,0.124326,0.104028,0.115657,0.14621,0.140078,0.0860556,0.0447193,0.0076118,46.0,NaN


In [219]:
final4_df = final3_df. merge(military, how="left", on="county")
final4_df.head()

,county,% lt_10,% 10_19,% 20_29,% 30_39,% 40_49,% 50-59,% 60_69,% 70_79,% 80_89,% over_90,nursing home capacity,prison capacity and jail employees,military installation
0,Anderson,0.0986114,0.112464,0.138999,0.167924,0.15124,0.126028,0.101392,0.0671934,0.0300955,0.006053,639.0,17730.0,NaN
1,Andrews,0.169665,0.162972,0.155905,0.153282,0.111789,0.102366,0.0766142,0.0403148,0.0221651,0.00492558,90.0,NaN,NaN
2,Angelina,0.138667,0.145997,0.134065,0.123758,0.119046,0.123365,0.104519,0.0703589,0.0330254,0.0071984,920.0,1399.0,NaN
3,Aransas,0.101838,0.10247,0.111533,0.105674,0.0978334,0.130374,0.155792,0.132735,0.0523942,0.00935761,212.0,NaN,NaN
4,Archer,0.106565,0.124749,0.124326,0.104028,0.115657,0.14621,0.140078,0.0860556,0.0447193,0.0076118,46.0,NaN,NaN


In [220]:
final5_df = final4_df.merge(hos_beds_df, how="left", on='county')
final5_df

,county,% lt_10,% 10_19,% 20_29,% 30_39,% 40_49,% 50-59,% 60_69,% 70_79,% 80_89,% over_90,nursing home capacity,prison capacity and jail employees,military installation,hospital beds
0,Anderson,0.0986114,0.112464,0.138999,0.167924,0.15124,0.126028,0.101392,0.0671934,0.0300955,0.006053,639.0,17730.0,NaN,156.0
1,Andrews,0.169665,0.162972,0.155905,0.153282,0.111789,0.102366,0.0766142,0.0403148,0.0221651,0.00492558,90.0,NaN,NaN,34.0
2,Angelina,0.138667,0.145997,0.134065,0.123758,0.119046,0.123365,0.104519,0.0703589,0.0330254,0.0071984,920.0,1399.0,NaN,470.0
3,Aransas,0.101838,0.10247,0.111533,0.105674,0.0978334,0.130374,0.155792,0.132735,0.0523942,0.00935761,212.0,NaN,NaN,NaN
4,Archer,0.106565,0.124749,0.124326,0.104028,0.115657,0.14621,0.140078,0.0860556,0.0447193,0.0076118,46.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,Wood,0.108236,0.112971,0.104168,0.0978771,0.0944537,0.130866,0.153229,0.130177,0.0569968,0.0110259,615.0,772.0,NaN,50.0
250,Yoakum,0.178651,0.167787,0.134633,0.123096,0.101703,0.114023,0.0875896,0.0581317,0.0281138,0.0062724,60.0,NaN,NaN,24.0
251,Young,0.136209,0.121399,0.109832,0.116156,0.105346,0.125399,0.132858,0.094103,0.0468083,0.0118912,335.0,NaN,NaN,67.0
252,Zapata,0.184873,0.168034,0.13721,0.126151,0.112808,0.0993935,0.0823403,0.0603639,0.0224046,0.00642169,59.0,NaN,NaN,NaN


In [221]:
final6_df = final5_df.merge(health_data, how="left", on="county")
final6_df

,county,% lt_10,% 10_19,% 20_29,% 30_39,% 40_49,% 50-59,% 60_69,% 70_79,% 80_89,...,Years of Potential Life Lost Rate,% Fair or Poor Health,% Smokers,% Physically Inactive,% With Access to Exercise Opportunities,% Flu Vaccinated,High School Graduation Rate,Income inequality,Social Association Rate,Average Daily PM2.5
0,Anderson,0.0986114,0.112464,0.138999,0.167924,0.15124,0.126028,0.101392,0.0671934,0.0300955,...,10927.0,20,17,23,26,43.0,97.0,4.3,10.0,9.9
1,Andrews,0.169665,0.162972,0.155905,0.153282,0.111789,0.102366,0.0766142,0.0403148,0.0221651,...,7138.0,20,13,26,94,24.0,96.0,4.3,7.3,7.3
2,Angelina,0.138667,0.145997,0.134065,0.123758,0.119046,0.123365,0.104519,0.0703589,0.0330254,...,8914.0,21,16,35,65,48.0,92.0,4.4,11.3,9.8
3,Aransas,0.101838,0.10247,0.111533,0.105674,0.0978334,0.130374,0.155792,0.132735,0.0523942,...,8770.0,21,14,36,81,42.0,89.0,5.3,5.1,9.3
4,Archer,0.106565,0.124749,0.124326,0.104028,0.115657,0.14621,0.140078,0.0860556,0.0447193,...,8253.0,14,14,19,23,46.0,99.0,4.4,9.1,8.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,Wood,0.108236,0.112971,0.104168,0.0978771,0.0944537,0.130866,0.153229,0.130177,0.0569968,...,8653.0,16,14,32,39,40.0,95.0,4.6,14.0,10.1
250,Yoakum,0.178651,0.167787,0.134633,0.123096,0.101703,0.114023,0.0875896,0.0581317,0.0281138,...,7149.0,22,14,28,67,17.0,98.0,4.8,8.2,7.3
251,Young,0.136209,0.121399,0.109832,0.116156,0.105346,0.125399,0.132858,0.094103,0.0468083,...,10209.0,18,16,29,60,38.0,97.0,4.2,12.2,8.2
252,Zapata,0.184873,0.168034,0.13721,0.126151,0.112808,0.0993935,0.0823403,0.0603639,0.0224046,...,7996.0,36,17,20,45,32.0,90.0,5.4,2.1,7.9


In [222]:
final6_df.isna().sum()

county                                       0
% lt_10                                      0
% 10_19                                      0
% 20_29                                      0
% 30_39                                      0
% 40_49                                      0
% 50-59                                      0
% 60_69                                      0
% 70_79                                      0
% 80_89                                      0
% over_90                                    0
nursing home capacity                       37
prison capacity and jail employees         196
military installation                      247
hospital beds                               69
Life Expectancy                             17
Age-Adjusted Death Rate                     10
Motor Vehicle Mortality Rate                51
% Insufficient Sleep                         0
% Enrolled in Free or Reduced Lunch          1
Segregation Index                           47
% Homeowners 

In [223]:
# filling in NaNs with 0s for columns where a NaN represents 0 
final6_df.loc[:, ["nursing home capacity", "prison capacity and jail employees", "military installation", "hospital beds" ]] = final6_df.loc[:,["nursing home capacity", "prison capacity and jail employees", "military installation", "hospital beds" ]].fillna(0)

In [224]:
final6_df.isna().sum()

county                                      0
% lt_10                                     0
% 10_19                                     0
% 20_29                                     0
% 30_39                                     0
% 40_49                                     0
% 50-59                                     0
% 60_69                                     0
% 70_79                                     0
% 80_89                                     0
% over_90                                   0
nursing home capacity                       0
prison capacity and jail employees          0
military installation                       0
hospital beds                               0
Life Expectancy                            17
Age-Adjusted Death Rate                    10
Motor Vehicle Mortality Rate               51
% Insufficient Sleep                        0
% Enrolled in Free or Reduced Lunch         1
Segregation Index                          47
% Homeowners                      

In [225]:
# filling in the remainder of the NaNs with the median of the column. 
#final6_df.iloc[:, 15:] = final6_df.iloc[:, 15:].apply(lambda col: col.fillna(col.median()), axis=0)

In [226]:
final6_df.isna().sum()

county                                      0
% lt_10                                     0
% 10_19                                     0
% 20_29                                     0
% 30_39                                     0
% 40_49                                     0
% 50-59                                     0
% 60_69                                     0
% 70_79                                     0
% 80_89                                     0
% over_90                                   0
nursing home capacity                       0
prison capacity and jail employees          0
military installation                       0
hospital beds                               0
Life Expectancy                            17
Age-Adjusted Death Rate                    10
Motor Vehicle Mortality Rate               51
% Insufficient Sleep                        0
% Enrolled in Free or Reduced Lunch         1
Segregation Index                          47
% Homeowners                      

In [227]:
# removing columns already in dataset
final6_df.drop(columns=["% Fair or Poor Health", "% Smokers", "% Physically Inactive", "Average Daily PM2.5", "Social Association Rate", "High School Graduation Rate"], inplace=True)

In [228]:
final6_df

,county,% lt_10,% 10_19,% 20_29,% 30_39,% 40_49,% 50-59,% 60_69,% 70_79,% 80_89,...,Age-Adjusted Death Rate,Motor Vehicle Mortality Rate,% Insufficient Sleep,% Enrolled in Free or Reduced Lunch,Segregation Index,% Homeowners,Years of Potential Life Lost Rate,% With Access to Exercise Opportunities,% Flu Vaccinated,Income inequality
0,Anderson,0.0986114,0.112464,0.138999,0.167924,0.15124,0.126028,0.101392,0.0671934,0.0300955,...,569.0,20.0,35,59.0,31.0,71,10927.0,26,43.0,4.3
1,Andrews,0.169665,0.162972,0.155905,0.153282,0.111789,0.102366,0.0766142,0.0403148,0.0221651,...,372.0,39.0,32,40.0,10.0,73,7138.0,94,24.0,4.3
2,Angelina,0.138667,0.145997,0.134065,0.123758,0.119046,0.123365,0.104519,0.0703589,0.0330254,...,462.0,19.0,35,67.0,37.0,66,8914.0,65,48.0,4.4
3,Aransas,0.101838,0.10247,0.111533,0.105674,0.0978334,0.130374,0.155792,0.132735,0.0523942,...,444.0,20.0,30,96.0,21.0,78,8770.0,81,42.0,5.3
4,Archer,0.106565,0.124749,0.124326,0.104028,0.115657,0.14621,0.140078,0.0860556,0.0447193,...,384.0,29.0,29,29.0,27.0,84,8253.0,23,46.0,4.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,Wood,0.108236,0.112971,0.104168,0.0978771,0.0944537,0.130866,0.153229,0.130177,0.0569968,...,423.0,32.0,29,58.0,44.0,80,8653.0,39,40.0,4.6
250,Yoakum,0.178651,0.167787,0.134633,0.123096,0.101703,0.114023,0.0875896,0.0581317,0.0281138,...,373.0,46.0,31,56.0,11.0,79,7149.0,67,17.0,4.8
251,Young,0.136209,0.121399,0.109832,0.116156,0.105346,0.125399,0.132858,0.094103,0.0468083,...,509.0,24.0,29,57.0,36.0,78,10209.0,60,38.0,4.2
252,Zapata,0.184873,0.168034,0.13721,0.126151,0.112808,0.0993935,0.0823403,0.0603639,0.0224046,...,401.0,16.0,35,83.0,12.0,70,7996.0,45,32.0,5.4


In [229]:
final6_df.head()

,county,% lt_10,% 10_19,% 20_29,% 30_39,% 40_49,% 50-59,% 60_69,% 70_79,% 80_89,...,Age-Adjusted Death Rate,Motor Vehicle Mortality Rate,% Insufficient Sleep,% Enrolled in Free or Reduced Lunch,Segregation Index,% Homeowners,Years of Potential Life Lost Rate,% With Access to Exercise Opportunities,% Flu Vaccinated,Income inequality
0,Anderson,0.0986114,0.112464,0.138999,0.167924,0.15124,0.126028,0.101392,0.0671934,0.0300955,...,569.0,20.0,35,59.0,31.0,71,10927.0,26,43.0,4.3
1,Andrews,0.169665,0.162972,0.155905,0.153282,0.111789,0.102366,0.0766142,0.0403148,0.0221651,...,372.0,39.0,32,40.0,10.0,73,7138.0,94,24.0,4.3
2,Angelina,0.138667,0.145997,0.134065,0.123758,0.119046,0.123365,0.104519,0.0703589,0.0330254,...,462.0,19.0,35,67.0,37.0,66,8914.0,65,48.0,4.4
3,Aransas,0.101838,0.10247,0.111533,0.105674,0.0978334,0.130374,0.155792,0.132735,0.0523942,...,444.0,20.0,30,96.0,21.0,78,8770.0,81,42.0,5.3
4,Archer,0.106565,0.124749,0.124326,0.104028,0.115657,0.14621,0.140078,0.0860556,0.0447193,...,384.0,29.0,29,29.0,27.0,84,8253.0,23,46.0,4.4


In [230]:
final6_df.to_csv("AJ_secondset.csv", index=False)

In [231]:
# merging the first and second sets of features together 
all_col_df = final_df_fillna.merge(final6_df, how="left", left_on="County", right_on="county")

In [232]:
all_col_df.shape

(254, 76)

In [233]:
all_col_df.columns

Index(['Median Age', '% Age 17 and Under', '% Age 65 and Older',
       '% Age 85 and Older', '% High School Graduate or Higher',
       '% Bachelor s Degree or Higher', 'Unemployment Rate (%)',
       'Ethnicity: % Hispanic', 'Race: % White Alone',
       'Race: % African American Alone',
       'Race: % American Indian & Alaska Native Alone', 'Race: % Asian Alone',
       'Race: % Native Hawaiian and Other Pacific Islander Alone',
       'Race: % Multi-Racial', 'Votes', 'Percent_Voted', 'Republicans 2016',
       'Democrats 2016', 'Green 2016', 'Libertarians 2016',
       'Preschool.Enrollment.Ratio.enrolled.ages.3.and.4', 'Gini.Coefficient',
       'Management.professional.and.related.occupations',
       'Service.occupations', 'Sales.and.office.occupations',
       'Farming.fishing.and.forestry.occupations',
       'Construction.extraction.maintenance.and.repair.occupations',
       'Production.transportation.and.material.moving.occupations',
       'Low.birthweight', 'Teen.births'

In [234]:
# removing redundent columns 
all_col_df.drop(columns=["county", '% Age 17 and Under', '% Age 65 and Older',
       '% Age 85 and Older'], inplace=True)

In [235]:
in_data_key = 'cleaned_data_1.csv'
data_location = 's3://{}/{}'.format(input_bucket, in_data_key)
all_col_df.to_csv(data_location, index=False)

In [236]:
df = pd.read_csv(data_location)

In [237]:
df.head()

,Median Age,% High School Graduate or Higher,% Bachelor s Degree or Higher,Unemployment Rate (%),Ethnicity: % Hispanic,Race: % White Alone,Race: % African American Alone,Race: % American Indian & Alaska Native Alone,Race: % Asian Alone,Race: % Native Hawaiian and Other Pacific Islander Alone,...,Age-Adjusted Death Rate,Motor Vehicle Mortality Rate,% Insufficient Sleep,% Enrolled in Free or Reduced Lunch,Segregation Index,% Homeowners,Years of Potential Life Lost Rate,% With Access to Exercise Opportunities,% Flu Vaccinated,Income inequality
0,39.1,80.2,11.8,3.6,17.82,75.11,21.49,0.68,0.90,0.14,...,569.0,20.0,35,59.0,31.0,71,10927.0,26,43.0,4.3
1,31.8,73.8,10.6,3.1,56.19,94.22,1.94,1.47,0.75,0.02,...,372.0,39.0,32,40.0,10.0,73,7138.0,94,24.0,4.3
2,37.1,79.9,15.7,5.1,22.18,81.27,15.44,0.75,1.17,0.06,...,462.0,19.0,35,67.0,37.0,66,8914.0,65,48.0,4.4
3,49.3,83.2,20.2,6.8,27.66,93.05,1.79,1.24,1.93,0.08,...,444.0,20.0,30,96.0,21.0,78,8770.0,81,42.0,5.3
4,44.3,90.1,21.8,3.4,8.68,95.58,1.03,1.42,0.43,0.03,...,384.0,29.0,29,29.0,27.0,84,8253.0,23,46.0,4.4
